## Salva frames vídeo
Salva os frames de um vídeo em uma pasta.

In [1]:
import os

import cv2

In [4]:
cap = cv2.VideoCapture('camera.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)

29.97002997002997


In [5]:
video_path = 'camera.mp4'  # Video's path
output_folder = 'saved_frames'  # Folder to save frames
frame_interval = 90  # Save every N frames

os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise ValueError(f'Error: Could not open video: {video_path}')

frame_count = 0
while True:  
    ret, frame = cap.read()
    if not ret:
        break
    if frame_count % frame_interval == 0:
        cv2.imwrite(f'{output_folder}/{frame_count}.png', frame)
    frame_count += 1

cap.release()
print(f"Done. Frames saved to '{output_folder}'.")


Done. Frames saved to 'saved_frames'.


## Realizar pré-anotação

In [6]:
from ultralytics import YOLO

In [7]:
model = YOLO('yolov8x')

In [8]:
model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [10]:
outputs = model.predict('saved_frames',
                        conf=0.3,
                        iou=0.3,
                        save=False,
                        classes=[2, 3, 5, 7], # car, motorcycle, bus, truck
                        save_txt=True,
                        verbose=False,
                        stream=False
                       )

Results saved to runs/detect/predict
465 labels saved to runs/detect/predict/labels


## Modifica ID das classes nos .txt (YOLO detection annotation)

In [11]:
import os
from pathlib import Path

In [12]:
path = 'runs/detect/predict/labels'
output_path = 'label_predict'

Path(output_path).mkdir(parents=True, exist_ok=True)

for filename in os.listdir(path):
    file = open(f'{path}/{filename}', 'r')
    annotation = ''
    for line in file.readlines():
        if line.startswith('2 '):
            line = f'0 {line[2:]}'
            annotation += line
        elif line.startswith('3 '):
            line = f'1 {line[2:]}'
            annotation += line
        elif line.startswith('5 '):
            line = f'2 {line[2:]}'
            annotation += line
        elif line.startswith('7 '):
            line = f'3 {line[2:]}'
            annotation += line
    if annotation:
        new_file = open(f'{output_path}/{filename}', 'w')
        new_file.write(annotation)
    new_file.close()
    file.close()
    #break

In [16]:
path = 'dataset/test/labels'
output_path = 'label_test'

Path(output_path).mkdir(parents=True, exist_ok=True)

for filename in os.listdir(path):
    file = open(f'{path}/{filename}', 'r')
    annotation = ''
    for line in file.readlines():
        if line.startswith('0 '):
            line = f'2 {line[2:]}'
            annotation += line
        elif line.startswith('1 '):
            line = f'3 {line[2:]}'
            annotation += line
        elif line.startswith('2 '):
            line = f'5 {line[2:]}'
            annotation += line
        elif line.startswith('3 '):
            line = f'7 {line[2:]}'
            annotation += line
    if annotation:
        new_file = open(f'{output_path}/{filename}', 'w')
        new_file.write(annotation)
    new_file.close()
    file.close()
    #break

### Remove anotações com IoU muito alto (duas detecções para o mesmo objeto)

In [13]:
import os
from pathlib import Path

input_path = 'label_predict'
output_path = 'label_predict_filtered'
max_iou = 0.9  # Higher will be removed

Path(output_path).mkdir(parents=True, exist_ok=True)

def compute_iou(box1, box2):
    # YOLO format: class x_center y_center width height
    _, x1, y1, w1, h1 = box1
    _, x2, y2, w2, h2 = box2

    # Convert to (x1, y1, x2, y2)
    xa1, ya1 = x1 - w1/2, y1 - h1/2
    xa2, ya2 = x1 + w1/2, y1 + h1/2
    xb1, yb1 = x2 - w2/2, y2 - h2/2
    xb2, yb2 = x2 + w2/2, y2 + h2/2

    inter_x1 = max(xa1, xb1)
    inter_y1 = max(ya1, yb1)
    inter_x2 = min(xa2, xb2)
    inter_y2 = min(ya2, yb2)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)

    area_a = (xa2 - xa1) * (ya2 - ya1)
    area_b = (xb2 - xb1) * (yb2 - yb1)
    union_area = area_a + area_b - inter_area

    return inter_area / union_area if union_area > 0 else 0



for filename in os.listdir(input_path):
    filepath = os.path.join(input_path, filename)
    with open(filepath, 'r') as f:
        lines = [list(map(float, l.strip().split())) for l in f if l.strip()]

    keep = []
    for i, box_i in enumerate(lines):
        should_keep = True
        for box_j in keep:
            if compute_iou(box_i, box_j) > max_iou:
                should_keep = False
                break
        if should_keep:
            keep.append(box_i)

    # Always save the file, even if empty
    outpath = os.path.join(output_path, filename)
    with open(outpath, 'w') as f:
        for box in keep:
            class_id = int(box[0])
            rest = box[1:]
            f.write(f'{class_id} ' + ' '.join(f'{v:.6f}' for v in rest) + '\n')
